In this notebook we tried to build an LDA model on dataset. We assume the dataset is preprocessed: It was removed stop words and contains only meaningful contents. There are only two columns of the input dataset:

In [1]:
import os
import pandas as pd 
import numpy as np 
import random
import re
import matplotlib as plt
import seaborn as sns
import gensim # this is LDA package

In [2]:
from gensim import corpora, models, similarities

In [ ]:
# This block read dataset


df = 